In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
#tf.debugging.set_log_device_placement(True)
#gpus = tf.config.experimental.list_physical_devices('GPU')
#print(len(gpus))

In [26]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [27]:
#data load, F.txt 파일이 있는 절대 경로를 입력하시면 됩니다.
#data 확인을 위하여 출력해봅니다. column name에 데이터가 들어가있어서 처리해주어야 합니다.
# N = 5
data = pd.read_csv('C:/Users/Administrator/Documents/MATLAB/Data set/F.txt', sep = '\t')
data

,0.07770158,-0.13003277,0.26838016,0.17627985,-0.04593009,0.09142768,0.10492659,0.14273107,0.09127340,0.11437427,...,0.07962369,-0.01639117,0.08893843,0.00350364,0.12996323,0.22611569,-0.12767046,0.10300189,0.17066372,0.13421567
0,-0.011763,0.00137,-0.000427,-0.010467,0.012151,-0.01357,0.007533,-0.00315,-0.00961,-0.005346,...,-0.008841,0.005438,-0.000925,0.012573,-0.003485,-0.000879,0.009161,0.010373,0.001752,-0.009546


In [29]:
arr = []
for i in range(len(data.iloc[0,:])):
    arr.append(i+1)

In [30]:
#load data
#column name에 data가 들어가있기 때문에 "names = arr" 로 처리해 줍니다. arr은 1 ~ len(F)의 숫자가 담겨있습니다.
dataF = pd.read_csv('C:/Users/Administrator/Documents/MATLAB/Data set/F.txt', sep = '\t', names = arr)
dataQ = pd.read_csv('C:/Users/Administrator/Documents/MATLAB/Data set/Q.txt', sep = '\t', names = arr)
data_nF = pd.read_csv('C:/Users/Administrator/Documents/MATLAB/Data set/nF.txt', sep = '\t', names = arr)

In [31]:
def make_arr(A):
    arA = []
    for j in range(len(A.iloc[0, :])):
        arA1 = []
        for i in range(len(A.iloc[:,0])):
            tmpA = A.iloc[:,j][i]
            arA1.append(tmpA)
        arA.append(arA1)
    return arA

In [32]:
#make array
arrF = make_arr(dataF)
dataF = np.array(arrF)

arrQ = make_arr(dataQ)
dataQ = np.array(arrQ)

arr_nF = make_arr(data_nF)
data_nF = np.array(arr_nF)

In [33]:
# N = 5
A = []
for n in range(1,6):
    x = np.exp(-(n**2))
    A.append(x)
A = np.diag(A)


In [34]:
# data 분할

import math
train_size = math.floor(len(dataF)*0.6) # train : 60%
val_size = math.floor(len(dataF)*0.2) #val : 20%
test_size = math.floor(len(dataF)*0.2) #test : 20%
#generate F_data, F_val, F_test
F_data = dataF[:train_size, :]
F_val = dataF[train_size:(val_size + train_size), :]
F_test = dataF[(val_size + train_size):(val_size + train_size + test_size), :]

#generate Q_data, Q_val, Q_test
Q_data = dataQ[:train_size, :]
Q_val = dataQ[train_size:(val_size + train_size), :]
Q_test = dataQ[(val_size + train_size):(val_size + train_size + test_size), :]

#generate nF_data, nF_val, nF_test
nF_data = data_nF[:train_size, :]
nF_val = data_nF[train_size:(val_size + train_size), :]
nF_test = data_nF[(val_size + train_size):(val_size + train_size + test_size), :]

print(f'{F_data.shape}, {F_test.shape}, {F_val.shape}')
print(f'{Q_data.shape}, {Q_test.shape}, {Q_val.shape}')
print(f'{nF_data.shape}, {nF_test.shape}, {nF_val.shape}')

(60000, 2), (20000, 2), (20000, 2)
(60000, 2), (20000, 2), (20000, 2)
(60000, 2), (20000, 2), (20000, 2)


In [38]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential() #Sequentioal

model.add(keras.layers.Dense(input_dim = 2, units = 10, activation= "tanh",kernel_initializer ="HeNormal" )) 
model.add(keras.layers.Dense(20, activation= "tanh")) 
model.add(keras.layers.Dense(50, activation= "tanh"))  
model.add(keras.layers.Dense(100, activation= "tanh")) 
model.add(keras.layers.Dense(120, activation= "tanh")) 
model.add(keras.layers.Dense(50, activation= "tanh"))
model.add(keras.layers.Dense(2, activation= "tanh"))

In [39]:
with tf.device('/GPU:0'):
    F_train = tf.constant(F_data)
    Q_train = tf.constant(Q_data)
    nF_train = tf.constant(nF_data)

In [41]:
model.compile(loss = "mse",
              optimizer = "ADAM",
              metrics = ["accuracy"])
history = model.fit(x = F_train, y = Q_train, validation_data=(F_val, Q_val),epochs = 50)

Epoch 1/50
1875/1875 [==============================] - 1s 481us/step - loss: 3.3644e-04 - accuracy: 0.9906 - val_loss: 3.3327e-04 - val_accuracy: 0.9908
Epoch 2/50
1875/1875 [==============================] - 1s 464us/step - loss: 3.1978e-04 - accuracy: 0.9913 - val_loss: 4.6369e-04 - val_accuracy: 0.9863
Epoch 3/50
1875/1875 [==============================] - 1s 467us/step - loss: 3.2122e-04 - accuracy: 0.9911 - val_loss: 2.6021e-04 - val_accuracy: 0.9924
Epoch 4/50
1875/1875 [==============================] - 1s 461us/step - loss: 3.1749e-04 - accuracy: 0.9911 - val_loss: 3.1363e-04 - val_accuracy: 0.9925
Epoch 5/50
1875/1875 [==============================] - 1s 467us/step - loss: 3.1207e-04 - accuracy: 0.9909 - val_loss: 3.1763e-04 - val_accuracy: 0.9916
Epoch 6/50
1875/1875 [==============================] - 1s 460us/step - loss: 3.0891e-04 - accuracy: 0.9910 - val_loss: 2.8009e-04 - val_accuracy: 0.9919
Epoch 7/50
1875/1875 [==============================] - 1s 460us/step - loss

In [42]:
#F 학습 후 F accuracy
result = model.evaluate(F_test, Q_test)

625/625 [==============================] - 0s 254us/step - loss: 1.6590e-05 - accuracy: 0.9984


In [43]:
#F 학습 후 nF accuracy
result = model.evaluate(nF_test, Q_test)

625/625 [==============================] - 0s 253us/step - loss: 0.0011 - accuracy: 0.9840


In [44]:
model.compile(loss = "mse",
              optimizer = "adam",
              metrics = ["accuracy"])
history = model.fit(x = nF_train, y = Q_train, validation_data=(nF_val, Q_val),epochs = 50)

Epoch 1/50
1875/1875 [==============================] - 1s 495us/step - loss: 0.0012 - accuracy: 0.9824 - val_loss: 0.0012 - val_accuracy: 0.9822
Epoch 2/50
1875/1875 [==============================] - 1s 468us/step - loss: 0.0012 - accuracy: 0.9821 - val_loss: 0.0011 - val_accuracy: 0.9825
Epoch 3/50
1875/1875 [==============================] - 1s 468us/step - loss: 0.0012 - accuracy: 0.9831 - val_loss: 0.0011 - val_accuracy: 0.9841
Epoch 4/50
1875/1875 [==============================] - 1s 471us/step - loss: 0.0012 - accuracy: 0.9823 - val_loss: 0.0011 - val_accuracy: 0.9840
Epoch 5/50
1875/1875 [==============================] - 1s 472us/step - loss: 0.0012 - accuracy: 0.9827 - val_loss: 0.0011 - val_accuracy: 0.9837
Epoch 6/50
1875/1875 [==============================] - 1s 467us/step - loss: 0.0012 - accuracy: 0.9824 - val_loss: 0.0012 - val_accuracy: 0.9821
Epoch 7/50
1875/1875 [==============================] - 1s 472us/step - loss: 0.0011 - accuracy: 0.9826 - val_loss: 0.0011 -

In [45]:
#nF 학습 후 nF accuracy
result = model.evaluate(nF_test, Q_test)

625/625 [==============================] - 0s 259us/step - loss: 0.0010 - accuracy: 0.9832
